#Install Dependencies & Run a SparkSession


In [0]:
#install pyspark
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=4825052dea65ca5dff007a3446e04661d0b87cecabe9c2ae3381c1e61e52c1c1
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [0]:
#create a sparksession
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName("AdmissionPredection").getOrCreate()

#Clone & Explore dataset

In [0]:
#clone the dataset
! git clone https://github.com/education454/admission_dataset

Cloning into 'admission_dataset'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), 5.60 KiB | 5.60 MiB/s, done.


In [0]:
#check the presence of dataset
! ls admission_dataset

Admission_Predict_Ver1.1.csv


In [0]:
#create a spark dataframe
df = spark.read.csv('/content/admission_dataset/Admission_Predict_Ver1.1.csv',header=True,inferSchema=True)

In [0]:
#display dataframe
df.show()

+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|Serial No|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|        1|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|        2|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|        3|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|        4|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|        5|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|        6|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|        7|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|        8|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|        9

In [0]:
#get the no.of rows & columns
print(df.count(),len(df.columns))

500 9


In [0]:
#print schema
df.printSchema()

root
 |-- Serial No: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOEFL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [0]:
#get the summary statistics
df.describe().show()

+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|summary|        Serial No|         GRE Score|      TOEFL Score|University Rating|               SOP|               LOR|              CGPA|          Research|    Chance of Admit|
+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|  count|              500|               500|              500|              500|               500|               500|               500|               500|                500|
|   mean|            250.5|           316.472|          107.192|            3.114|             3.374|             3.484| 8.576440000000003|              0.56| 0.7217399999999996|
| stddev|144.4818327679989|11.295148372354712|6.081867659564538|1.143511800759815|0.9910036207566072|0.92

#Data Cleaning

In [0]:
#drop the unnecessary column
df = df.drop('Serial No')

In [0]:
#display the dataframe
df.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|      302|        102|                1|2.0|1.5| 8.0|       0|            0.5|
|      323|        108|                3

In [0]:
#check for null values
for i in df.columns:
  print(i+':',df[df[i].isNull()].count())

GRE Score: 0
TOEFL Score: 0
University Rating: 0
SOP: 0
LOR: 0
CGPA: 0
Research: 0
Chance of Admit: 0


#Correlation Analysis & Feature Selection

In [0]:
from os import stat
from pyspark.sql.functions import corr

# correlation analys

In [0]:
for col in df.columns:
  print('Correlation to chance of admit col for {} is {}'.format(col,df.stat.corr('Chance of Admit',col)))

Correlation to chance of admit col for GRE Score is 0.8103506354632598
Correlation to chance of admit col for TOEFL Score is 0.7922276143050823
Correlation to chance of admit col for University Rating is 0.6901323687886892
Correlation to chance of admit col for SOP is 0.6841365241316723
Correlation to chance of admit col for LOR is 0.6453645135280112
Correlation to chance of admit col for CGPA is 0.882412574904574
Correlation to chance of admit col for Research is 0.5458710294711379
Correlation to chance of admit col for Chance of Admit is 1.0


In [0]:
# feature selection
from pyspark.ml.feature import VectorAssembler
assenbler = VectorAssembler(inputCols=['GRE Score','TOEFL Score','CGPA'],outputCol='features')

In [0]:
#display dataframe
output_data = assenbler.transform(df)

output_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|          features|
+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|[337.0,118.0,9.65]|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|[324.0,107.0,8.87]|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72| [316.0,104.0,8.0]|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|[322.0,110.0,8.67]|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|[314.0,103.0,8.21]|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|[330.0,115.0,9.34]|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75| [321.0,109.0,8.2]|
|      308

# Build the Linear Regression Model

In [0]:
#import Linearregression and create final data
from pyspark.ml.regression import LinearRegression
final_data = output_data.select('features','Chance of Admit')

In [0]:
#print schema of final data
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [0]:
#split the dataset into training and testing set
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [0]:
#build & train the model
model = LinearRegression(featuresCol='features',labelCol='Chance of Admit')
model = model.fit(train_data)

In [0]:
#get coefficients & intercept
print('Coefficient = ',model.coefficients)
print('intercept = ',model.intercept)

Coefficient =  [0.002072187852978751,0.0035920164942791384,0.14838170483319102]
intercept =  -1.590346206069565


In [0]:
#get summary of the model
summary = model.summary

In [0]:
#print the rmse & r2 score
print('RMSE: ',summary.rootMeanSquaredError)
print('R2 : ',summary.r2)

RMSE:  0.06194198447821799
R2 :  0.8119446221572038


# Evaluate & Save the Model

In [0]:
#transform on the test data
predections = model.transform(test_data)

In [0]:
#display the predictions
predections.show(15)

+------------------+---------------+-------------------+
|          features|Chance of Admit|         prediction|
+------------------+---------------+-------------------+
| [295.0,99.0,7.57]|           0.37|0.49980834908005733|
| [295.0,99.0,7.65]|           0.57| 0.5116788854667125|
|  [296.0,97.0,7.8]|           0.49| 0.5288242960561116|
| [296.0,99.0,7.28]|           0.47| 0.4588498425314105|
| [297.0,96.0,7.89]|           0.43| 0.5406588208497982|
| [298.0,97.0,7.21]|           0.45| 0.4454234659104863|
|  [298.0,98.0,7.5]|           0.44| 0.4920461768063913|
| [298.0,99.0,7.46]|           0.53|0.48970292510734237|
|[298.0,100.0,7.95]|           0.58| 0.5660019769698852|
|[298.0,101.0,7.86]|           0.54| 0.5562396400291771|
|[299.0,102.0,8.62]|           0.56|   0.67467394004966|
| [300.0,95.0,8.22]|           0.62| 0.5922493305094088|
|[300.0,100.0,8.66]|           0.64| 0.6754973631074084|
|[300.0,101.0,7.88]|           0.59| 0.5633516498317983|
|[300.0,102.0,8.17]|           

In [0]:
#evaluate the model
from pyspark.ml.evaluation import RegressionEvaluator
evaluat = RegressionEvaluator(predictionCol='prediction',labelCol='Chance of Admit',metricName='r2')
print('r2 on the test data :',evaluat.evaluate(predections))

r2 on the test data : 0.7838243869536594


In [0]:
#save the model
model.save("AdmissionPredectionModel")

In [0]:
#load the model
from pyspark.ml.regression import LinearRegressionModel
loda_model = LinearRegressionModel.load('AdmissionPredectionModel')